In [15]:
from sqlalchemy import create_engine
import pandas as pd

path = "wjazzd.db" # REPLACE THIS WITH PATH TO FILE
engine = create_engine(f"sqlite:///{path}")

melody = pd.read_sql("melody", engine)
# melody_type = pd.read_sql("melody_type", engine)
beats = pd.read_sql("beats", engine)

In [30]:
#solo_info = pd.read_sql("solo_info", engine)
#solo_info

## Melody
All melodies are sequences of events and stored in a single table, where each row represents one melody event. Each event has an unique eventid and melid which is used to cross-reference across the database.

In [6]:
melody.columns

Index(['eventid', 'melid', 'onset', 'pitch', 'duration', 'period', 'division',
       'bar', 'beat', 'tatum', 'subtatum', 'num', 'denom', 'beatprops',
       'beatdur', 'tatumprops', 'f0_mod', 'loud_max', 'loud_med', 'loud_sd',
       'loud_relpos', 'loud_cent', 'loud_s2b', 'f0_range', 'f0_freq_hz',
       'f0_med_dev'],
      dtype='object')

In [78]:
pd.set_option('display.max_columns', None)
melody.head(20)

,eventid,melid,onset,pitch,duration,period,division,bar,beat,tatum,subtatum,num,denom,beatprops,beatdur,tatumprops,f0_mod,loud_max,loud_med,loud_sd,loud_relpos,loud_cent,loud_s2b,f0_range,f0_freq_hz,f0_med_dev
0,1,1,10.343492,65.0,0.138776,4,1,0,1,1,0,4,4,None,0.291746,"(1,)",,0.126209,66.526087,5.541147,0.307692,0.389466,1.056169,37.794261,12.932532,-0.328442
1,2,1,10.637642,63.0,0.171247,4,4,0,2,1,0,4,4,None,0.286621,"(1.0, 1.0, 1.0, 1.173)",,0.349751,69.133321,2.912412,0.250000,0.468687,1.120317,6.365930,6.956935,11.135423
2,3,1,10.843719,58.0,0.081270,4,4,0,2,4,0,4,4,None,0.286621,"(1.0, 1.0, 1.0, 1.173)",,0.094051,66.352130,3.564563,0.428571,0.531354,1.310389,68.010392,NaN,32.366787
3,4,1,10.948209,61.0,0.235102,4,1,0,3,1,0,4,4,None,0.298844,"(1,)",,0.521187,66.484173,2.414298,0.818182,0.559333,0.984047,15.443906,5.867151,-3.374696
4,5,1,11.232653,63.0,0.130612,4,1,0,4,1,0,4,4,None,0.297120,"(1,)",,0.560737,71.699054,2.185794,0.166667,0.438973,1.061262,11.444363,8.329975,6.377737
5,6,1,11.551927,58.0,0.188662,4,1,1,1,1,0,4,4,None,0.310023,"(1,)",,0.534657,67.636708,7.635221,0.411765,0.359536,1.049956,39.368720,6.589582,16.146429
6,7,1,11.859592,58.0,0.481814,4,1,1,2,1,0,4,4,None,0.305283,"(1,)",vibrato,0.584914,63.659343,5.518070,0.068182,0.403372,0.983151,39.429103,5.406750,11.239471
7,8,1,14.535692,50.0,0.159637,4,1,3,3,1,0,4,4,None,0.269977,"(1,)",,-0.129185,58.507975,5.020340,0.133333,0.368384,0.927912,174.398513,NaN,25.203232
8,9,1,14.799819,57.0,0.145125,4,2,3,4,1,0,4,4,None,0.303084,"(1.0, 0.438)",,0.599931,71.173670,2.938194,0.285714,0.551884,1.064195,27.066543,7.758283,25.736430
9,10,1,14.973968,60.0,0.110295,4,2,3,4,2,0,4,4,None,0.303084,"(1.0, 0.438)",,0.484532,69.632891,2.325457,0.600000,0.508617,1.038483,17.141304,11.184763,15.693739


## Beats
References `melody` table with `melid`.  Stores (tapped) beats along with chord, form, bass pitch and other annotations.

- **Onset**: (float) time where the beat begins (in seconds)
- **Bar**: (int) segment of time
- **Beat**: (int) each bar has X beats (determined by signature). In the `beats` table, we have exactly one event for each beat in each bar
- **Signature**: (text) number of beats per bar. Typically '4/4' (means 4 beats at each bar). Only one non-blank entry per `melid`
- **Chord**: (text) Accompaning chord. Only available in certain beats.
- **Form**: (text) Form part of this and subsequent beats (e.g., A1). Seems to be non-blank every 8 bars
- **Bass pitch**: bass pitch of the beat
- **Chorus ID**: chorus number of the beat

In [24]:
beats.columns

Index(['beatid', 'melid', 'onset', 'bar', 'beat', 'signature', 'chord', 'form',
       'bass_pitch', 'chorus_id'],
      dtype='object')

In [5]:
beats

,beatid,melid,onset,bar,beat,signature,chord,form,bass_pitch,chorus_id
0,1,1,9.171882,-1,1,,,I1,42.0,0
1,2,1,9.488254,-1,2,,,,42.0,0
2,3,1,9.779955,-1,3,,,,40.0,0
3,4,1,10.052608,-1,4,,,,40.0,0
4,5,1,10.339796,0,1,,Bb6,,50.0,0
...,...,...,...,...,...,...,...,...,...,...
132324,132325,456,64.713311,55,2,,,,NaN,2
132325,132326,456,65.005896,55,3,,,,NaN,2
132326,132327,456,65.289524,55,4,,,,NaN,2
132327,132328,456,65.573152,56,1,,NC,,NaN,2


## Analysis of melody 1

In [36]:
melody.loc[melody['melid'] == 1].head(20)

,eventid,melid,onset,pitch,duration,period,division,bar,beat,tatum,subtatum,num,denom,beatprops,beatdur,tatumprops,f0_mod,loud_max,loud_med,loud_sd,loud_relpos,loud_cent,loud_s2b,f0_range,f0_freq_hz,f0_med_dev
0,1,1,10.343492,65.0,0.138776,4,1,0,1,1,0,4,4,None,0.291746,"(1,)",,0.126209,66.526087,5.541147,0.307692,0.389466,1.056169,37.794261,12.932532,-0.328442
1,2,1,10.637642,63.0,0.171247,4,4,0,2,1,0,4,4,None,0.286621,"(1.0, 1.0, 1.0, 1.173)",,0.349751,69.133321,2.912412,0.250000,0.468687,1.120317,6.365930,6.956935,11.135423
2,3,1,10.843719,58.0,0.081270,4,4,0,2,4,0,4,4,None,0.286621,"(1.0, 1.0, 1.0, 1.173)",,0.094051,66.352130,3.564563,0.428571,0.531354,1.310389,68.010392,NaN,32.366787
3,4,1,10.948209,61.0,0.235102,4,1,0,3,1,0,4,4,None,0.298844,"(1,)",,0.521187,66.484173,2.414298,0.818182,0.559333,0.984047,15.443906,5.867151,-3.374696
4,5,1,11.232653,63.0,0.130612,4,1,0,4,1,0,4,4,None,0.297120,"(1,)",,0.560737,71.699054,2.185794,0.166667,0.438973,1.061262,11.444363,8.329975,6.377737
5,6,1,11.551927,58.0,0.188662,4,1,1,1,1,0,4,4,None,0.310023,"(1,)",,0.534657,67.636708,7.635221,0.411765,0.359536,1.049956,39.368720,6.589582,16.146429
6,7,1,11.859592,58.0,0.481814,4,1,1,2,1,0,4,4,None,0.305283,"(1,)",vibrato,0.584914,63.659343,5.518070,0.068182,0.403372,0.983151,39.429103,5.406750,11.239471
7,8,1,14.535692,50.0,0.159637,4,1,3,3,1,0,4,4,None,0.269977,"(1,)",,-0.129185,58.507975,5.020340,0.133333,0.368384,0.927912,174.398513,NaN,25.203232
8,9,1,14.799819,57.0,0.145125,4,2,3,4,1,0,4,4,None,0.303084,"(1.0, 0.438)",,0.599931,71.173670,2.938194,0.285714,0.551884,1.064195,27.066543,7.758283,25.736430
9,10,1,14.973968,60.0,0.110295,4,2,3,4,2,0,4,4,None,0.303084,"(1.0, 0.438)",,0.484532,69.632891,2.325457,0.600000,0.508617,1.038483,17.141304,11.184763,15.693739


In [51]:
beats.loc[beats['melid']==1].head(20)

,beatid,melid,onset,bar,beat,signature,chord,form,bass_pitch,chorus_id
0,1,1,9.171882,-1,1,,,I1,42.0,0
1,2,1,9.488254,-1,2,,,,42.0,0
2,3,1,9.779955,-1,3,,,,40.0,0
3,4,1,10.052608,-1,4,,,,40.0,0
4,5,1,10.339796,0,1,,Bb6,,50.0,0
5,6,1,10.631542,0,2,,,,50.0,0
6,7,1,10.918163,0,3,,,,50.0,0
7,8,1,11.217007,0,4,,,,42.0,0
8,9,1,11.514127,1,1,4/4,Bb6,A1,42.0,1
9,10,1,11.824150,1,2,,,,42.0,1


In [79]:
beats.loc[beats['chord']!=''].head(25)

,beatid,melid,onset,bar,beat,signature,chord,form,bass_pitch,chorus_id
4,5,1,10.339796,0,1,,Bb6,,50.0,0
8,9,1,11.514127,1,1,4/4,Bb6,A1,42.0,1
10,11,1,12.129433,1,3,,G7,,43.0,1
12,13,1,12.703628,2,1,,C-7,,39.0,1
14,15,1,13.290816,2,3,,F7,,41.0,1
16,17,1,13.894399,3,1,,Bb,,38.0,1
18,19,1,14.477234,3,3,,G-7,,41.0,1
20,21,1,15.050295,4,1,,C-7,,43.0,1
22,23,1,15.649977,4,3,,F7,,39.0,1
24,25,1,16.230317,5,1,,F-7,,34.0,1


In [54]:
bb = beats['bass_pitch']

In [60]:
bb.describe()

count    122540.000000
mean         41.449714
std           4.427586
min          28.000000
25%          38.000000
50%          42.000000
75%          45.000000
max          53.000000
Name: bass_pitch, dtype: float64

In [64]:
bb == 'nan'

0         False
1         False
2         False
3         False
4         False
          ...  
132324    False
132325    False
132326    False
132327    False
132328    False
Name: bass_pitch, Length: 132329, dtype: bool

### Chords
How many different chords are there? (chord vocab)

Currently we have 419 unique chords

For melody `melid=1` we have 13 unique chords

In [72]:
chords = beats.loc[beats['chord'] != '']

In [73]:
chords['chord'].describe()

count     30548
unique      419
top         Bb7
freq       2046
Name: chord, dtype: object

In [76]:
chords.loc[chords['melid']==1]['chord'].describe()

count     202
unique     13
top        F7
freq       40
Name: chord, dtype: object

### Beats